In [1]:
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.model_selection import train_test_split

In [2]:
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt

El fichero «edgar_allan_poe.txt» ya está ahí, no se recupera.

El fichero «robert_frost.txt» ya está ahí, no se recupera.



In [3]:
edp='edgar_allan_poe.txt'
rf='robert_frost.txt'

In [4]:
i_f= [edp,
     rf]

In [5]:
!head 'edgar_allan_poe.txt'

LO! Death hath rear'd himself a throne
In a strange city, all alone,
Far down within the dim west
Where the good, and the bad, and the worst, and the best,
Have gone to their eternal rest.
 
There shrines, and palaces, and towers
Are not like any thing of ours
Oh no! O no! ours never loom
To heaven with that ungodly gloom!


In [6]:
!head 'robert_frost.txt'

Two roads diverged in a yellow wood,
And sorry I could not travel both
And be one traveler, long I stood
And looked down one as far as I could
To where it bent in the undergrowth; 

Then took the other, as just as fair,
And having perhaps the better claim
Because it was grassy and wanted wear,
Though as for that the passing there


In [7]:
i_t= []
labels= []

for label, f in enumerate(i_f):
    print(f, 'correspond to label', label)\
    
    for line in open(f):
        line = line.rstrip().lower()
        if line:
            line = line.translate(str.maketrans('', '', string.punctuation))
            
            
            i_t.append(line)
            labels.append(label)
        
        

edgar_allan_poe.txt correspond to label 0
robert_frost.txt correspond to label 1


In [188]:
train_text, test_text, Ytrain, Ytest = train_test_split(i_t, labels)

In [9]:
len(Ytrain), len(Ytest)

(1615, 539)

In [10]:
train_text[:5]

['your destination and your destinys',
 'is supposed to be mad',
 'around and under',
 'for hog reeve in march meeting here in warren',
 'but though repeatedly he strove']

In [11]:
Ytrain[:5]

[1, 1, 1, 1, 1]

In [12]:
idx = 0
w2i = {'nn': 0}

In [13]:
for t in train_text:
    tokens = t.split()
    for token in tokens:
        if token not in w2i:
            idx +=1
            w2i[token]= idx
for y in test_text:
    tokens = y.split()
    for token in tokens:
        if token not in w2i:
            idx +=1
            w2i[token]= idx    

In [14]:
len(w2i)

2977

In [189]:
tti = []
testti = []

for q in train_text:
    tokens = q.split()
    lai = [w2i[token] for token in tokens]
    tti.append(lai)
    
for w in test_text:
    tokens = w.split()
    laii = [w2i[token] for token in tokens]
    testti.append(laii)


In [190]:
V= len(w2i)
#A es una matriz de probabilidad en base a pi y a si misma
#ejemplo= si despues de hola viene como, se agrega a la matriz probabilidad de que eso pase
A0 = np.ones((V, V))
pi0 = np.ones(V)

A1 = np.ones((V, V))
pi1 = np.ones(V)

In [191]:

def compute_counts(text_as_int, A, pi):
  for tokens in text_as_int:
    last_idx = None
    for idx in tokens:
      if last_idx is None:

        pi[idx] += 1
      else:

        A[last_idx, idx] += 1


      last_idx = idx


compute_counts([t for t, y in zip(tti, Ytrain) if y == 0], A0, pi0)
compute_counts([t for t, y in zip(tti, Ytrain) if y == 1], A1, pi1)

In [192]:
A0 /= A0.sum(axis=1, keepdims=True)
pi0 /= pi0.sum()

A1 /= A1.sum(axis=1, keepdims=True)
pi1 /= pi1.sum()

In [193]:
logA0 = np.log(A0)
logpi0 = np.log(pi0)

logA1 = np.log(A1)
logpi1 = np.log(pi1)
print(logpi1)

[-8.30721263 -7.61406545 -8.30721263 ... -8.30721263 -8.30721263
 -8.30721263]


In [194]:
count0 = sum(y == 0 for y in Ytrain)
count1 = sum(y == 1 for y in Ytrain)
total = len(Ytrain)
p0 = count0 / total
p1 = count1 / total
logp0 = np.log(p0)
logp1 = np.log(p1)


In [195]:
class classifier:
    def __init__(self, logAs, logpis, logpriors):
        self.logAs = logAs
        self.logpis = logpis
        self.logpriors = logpriors
        self.K = len(logpriors)
    def predict(self,logAs, logpis, inputs):
        predict=[]
        pred=[]
        for x in range(1):
            for q in inputs:
                indexprev = None
                prob1= 0
                prob2= 0
                for c in q:
                    if indexprev is None:
                        prob1+=logpis[1][c]
                        prob2+=logpis[0][c]
                    else:
                        prob1+=logAs[1][indexprev, c]
                        prob2+=logAs[0][indexprev, c]
                    indexprev= c
                pred.append([prob1,prob2])
            
        for y in range(len(pred)):
            if pred[y][0]> pred[y][1]:
                predict.append(1)
            else:
                predict.append(0)
        
        return predict
        
        

In [196]:
clf= classifier([logA0, logA1], [logpi0, logpi1], [logp0, logp1])

In [197]:
logAs= [logA0, logA1]
logpis= [logpi0, logpi1]
Ptrain= clf.predict(logAs, logpis, tti)
print(Ytrain)
print(Ptrain)

[0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 

In [199]:
from sklearn.metrics import confusion_matrix, f1_score

In [200]:
f1_score(Ytrain, Ptrain)

0.9986065954482118

In [202]:
Ptest= clf.predict(logAs, logpis, testti)

In [203]:
f1_score(Ytest, Ptest)

0.8642659279778393